In [1]:
# Import dependencies
import json
import os
import pandas as pd
from pprint import pprint
import numpy as np

In [2]:
# Read in world json data
with open('Data_Resources/Coordinate_Data/minified_world_data.json', 'r') as f:
    json_data = f.read()
    
json_d = json.loads(json_data)


## Temperature Data

In [3]:
# Read in temp data
temp_data_file = 'Data_Resources/Global Temperature Changes by Country/GlobalLandTemperaturesByCountry.csv'
temp_data_df = pd.read_csv(temp_data_file, parse_dates=True)
temp_data_df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,Country
0,1743-11-01,4.384,2.294,Åland
1,1743-12-01,NaN,NaN,Åland
2,1744-01-01,NaN,NaN,Åland
3,1744-02-01,NaN,NaN,Åland
4,1744-03-01,NaN,NaN,Åland


In [4]:
temp_data_df.count()
temp_data_df.dtypes

dt                                object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
Country                           object
dtype: object

In [5]:
temp_data_df = temp_data_df.fillna(0)
temp_data_df.count()
temp_data_df.dtypes

dt                                object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
Country                           object
dtype: object

In [6]:
# temp_data_df['AverageTemperature'] = pd.to_numeric(df['AverageTemperature'])

In [7]:
# Cleaning temp data - calculate yearly temp average per year (1950-2013)
temp_data_df['dt']=pd.to_datetime(temp_data_df['dt'])
temp_data_df['year'] = pd.PeriodIndex(temp_data_df.dt, freq='Y')
temp_data_df['year'].value_counts().sort_index() #Do i need sort_index()?
temp_data_df = temp_data_df.rename(columns={'year':'Year'})

yearly_average_temp = temp_data_df.groupby(['Country', 'Year'])['AverageTemperature'].mean()
yearly_average_temp = yearly_average_temp.reset_index()
yearly_average_temp

# yearly_average_temp_df = yearly_average_temp
# yearly_average_temp_df = 
# *** FILTER YEARS FOR 1950-2013 *** #

,Country,Year,AverageTemperature
0,Afghanistan,1838,14.295222
1,Afghanistan,1839,0.000000
2,Afghanistan,1840,12.295667
3,Afghanistan,1841,11.664667
4,Afghanistan,1842,11.366000
...,...,...,...
48238,Åland,2009,6.489083
48239,Åland,2010,4.861917
48240,Åland,2011,7.170750
48241,Åland,2012,6.063917


## Emissions Data

In [8]:
# Read in emissions data
emissions_data_file = 'Data_Resources/Emissions/annual-co-emissions-by-region.csv'
emissions_data_df = pd.read_csv(emissions_data_file)
emissions_data_df = emissions_data_df.rename(columns={'Entity':'Country'})
emissions_data_df.head()

,Country,Code,Year,Annual CO2 emissions
0,Afghanistan,AFG,1949,0.014656
1,Afghanistan,AFG,1950,0.084272
2,Afghanistan,AFG,1951,0.091600
3,Afghanistan,AFG,1952,0.091600
4,Afghanistan,AFG,1953,0.106256


In [9]:
emissions_data_df.count()

Country                 23372
Code                    19244
Year                    23372
Annual CO2 emissions    23372
dtype: int64

In [10]:
emissions_data_df = emissions_data_df.fillna(0)
emissions_data_df.count()
# emissions_data_df.dtypes

Country                 23372
Code                    23372
Year                    23372
Annual CO2 emissions    23372
dtype: int64

In [11]:
# Display emissions data
emissions_data_df.head()

,Country,Code,Year,Annual CO2 emissions
0,Afghanistan,AFG,1949,0.014656
1,Afghanistan,AFG,1950,0.084272
2,Afghanistan,AFG,1951,0.091600
3,Afghanistan,AFG,1952,0.091600
4,Afghanistan,AFG,1953,0.106256


## Join Data


In [12]:
yearly_average_temp['Year'] = yearly_average_temp['Year'].astype(str).astype(int)
#yearly_average_temp['Year'].map(lambda x:int(x))

In [13]:
comb = yearly_average_temp.merge(emissions_data_df, on=['Year', 'Country'])
# comb  = comb.loc[comb['Year'], comb['Year']
                 
comb = comb.loc[(comb["Year"]>=1950)&(comb["Year"]<=2013)]    
comb

,Country,Year,AverageTemperature,Code,Annual CO2 emissions
1,Afghanistan,1950,13.043500,AFG,0.084272
2,Afghanistan,1951,13.967750,AFG,0.091600
3,Afghanistan,1952,14.175417,AFG,0.091600
4,Afghanistan,1953,14.650750,AFG,0.106256
5,Afghanistan,1954,13.691333,AFG,0.106256
...,...,...,...,...,...
16919,Zimbabwe,2009,21.377250,ZWE,5.519457
16920,Zimbabwe,2010,21.986250,ZWE,7.706701
16921,Zimbabwe,2011,21.602417,ZWE,9.498475
16922,Zimbabwe,2012,21.521333,ZWE,7.695208


In [14]:
year_list = comb['Year'].nunique()
year_max = comb['Year'].max()
year_min = comb['Year'].min()
year_max
year_min

1950

In [15]:
for country in json_d['features']:
    country_df = comb.loc[yearly_average_temp['Country'] == country['properties']['name']].copy()
    
    years = country_df['Year'].unique()
    
    for i in years:
#       print(country[0][f'year_temp{i}'])
        country['properties'][f'year_temp_{i}'] = country_df.loc[country_df['Year'] == i, 'AverageTemperature'].values[0]
        country['properties'][f'year_emission_{i}'] = country_df.loc[country_df['Year'] == i, 'Annual CO2 emissions'].values[0]
        

In [16]:
[pprint(country['properties']) for country in json_d['features']]

{'name': 'Afghanistan',
 'year_emission_1950': 0.084272,
 'year_emission_1951': 0.0916,
 'year_emission_1952': 0.0916,
 'year_emission_1953': 0.10625599999999999,
 'year_emission_1954': 0.10625599999999999,
 'year_emission_1955': 0.15388800000000002,
 'year_emission_1956': 0.1832,
 'year_emission_1957': 0.29312,
 'year_emission_1958': 0.32976,
 'year_emission_1959': 0.38457142047244103,
 'year_emission_1960': 0.4138847238016301,
 'year_emission_1961': 0.4907977045669292,
 'year_emission_1962': 0.6885942727559057,
 'year_emission_1963': 0.7067359773228347,
 'year_emission_1964': 0.8385508296062993,
 'year_emission_1965': 1.0069165341732285,
 'year_emission_1966': 1.0911588182677163,
 'year_emission_1967': 1.2818651137007877,
 'year_emission_1968': 1.2233908463828889,
 'year_emission_1969': 0.9412319773228348,
 'year_emission_1970': 1.6703968442024748,
 'year_emission_1971': 1.8939314091338584,
 'year_emission_1972': 1.5311656932283466,
 'year_emission_1973': 1.6372446580680542,
 'year_e

 'year_emission_1955': 11.600224,
 'year_emission_1956': 11.834719999999999,
 'year_emission_1957': 11.728464,
 'year_emission_1958': 12.453936,
 'year_emission_1959': 0.098928,
 'year_emission_1960': 0.087936,
 'year_emission_1961': 0.087936,
 'year_emission_1962': 0.07328,
 'year_emission_1963': 0.07328,
 'year_emission_1964': 0.07328,
 'year_emission_1965': 0.087936,
 'year_emission_1966': 0.084272,
 'year_emission_1967': 0.0916,
 'year_emission_1968': 0.18320000000000006,
 'year_emission_1969': 0.18686400000000006,
 'year_emission_1970': 0.208848,
 'year_emission_1971': 0.18320000000000006,
 'year_emission_1972': 0.168544,
 'year_emission_1973': 0.161216,
 'year_emission_1974': 0.11358399999999999,
 'year_emission_1975': 0.102592,
 'year_emission_1976': 0.12824000000000002,
 'year_emission_1977': 0.12824000000000002,
 'year_emission_1978': 0.14656,
 'year_emission_1979': 0.102592,
 'year_emission_1980': 0.10625599999999999,
 'year_emission_1981': 0.135568,
 'year_emission_1982': 0.

 'year_temp_1981': 23.15458333333333,
 'year_temp_1982': 23.25550000000001,
 'year_temp_1983': 23.493333333333336,
 'year_temp_1984': 23.192000000000004,
 'year_temp_1985': 22.916583333333335,
 'year_temp_1986': 23.193249999999995,
 'year_temp_1987': 23.626583333333333,
 'year_temp_1988': 23.51908333333333,
 'year_temp_1989': 22.856000000000005,
 'year_temp_1990': 23.37958333333334,
 'year_temp_1991': 23.41241666666667,
 'year_temp_1992': 23.239500000000003,
 'year_temp_1993': 22.98875,
 'year_temp_1994': 23.20866666666667,
 'year_temp_1995': 23.396500000000003,
 'year_temp_1996': 23.358583333333332,
 'year_temp_1997': 23.423833333333334,
 'year_temp_1998': 23.689416666666663,
 'year_temp_1999': 23.446833333333334,
 'year_temp_2000': 23.762583333333335,
 'year_temp_2001': 23.6615,
 'year_temp_2002': 23.82325,
 'year_temp_2003': 23.857166666666668,
 'year_temp_2004': 23.778666666666666,
 'year_temp_2005': 23.89691666666667,
 'year_temp_2006': 23.77808333333333,
 'year_temp_2007': 23.724

 'year_emission_1977': 163.73194364155438,
 'year_emission_1978': 162.10393660738706,
 'year_emission_1979': 163.67326069683585,
 'year_emission_1980': 120.56447016722791,
 'year_emission_1981': 112.52856638171288,
 'year_emission_1982': 137.51497083298494,
 'year_emission_1983': 149.21343130339758,
 'year_emission_1984': 149.94267692349166,
 'year_emission_1985': 159.94710964633396,
 'year_emission_1986': 147.53604150823142,
 'year_emission_1987': 158.6073523330751,
 'year_emission_1988': 175.1077964761723,
 'year_emission_1989': 190.09800163069858,
 'year_emission_1990': 209.2383435071259,
 'year_emission_1991': 225.95422635147548,
 'year_emission_1992': 226.9570340554339,
 'year_emission_1993': 235.57662590543168,
 'year_emission_1994': 263.5203779056178,
 'year_emission_1995': 271.1475250280357,
 'year_emission_1996': 274.00773761302656,
 'year_emission_1997': 267.09927885526656,
 'year_emission_1998': 306.00628146885884,
 'year_emission_1999': 379.62703513918274,
 'year_emission_2

 'year_emission_1961': 13.335483148346455,
 'year_emission_1962': 14.060390546141731,
 'year_emission_1963': 14.965279682519686,
 'year_emission_1964': 16.170319659842523,
 'year_emission_1965': 16.389921932598426,
 'year_emission_1966': 19.54007045543307,
 'year_emission_1967': 19.30423723968504,
 'year_emission_1968': 21.113688637480315,
 'year_emission_1969': 22.193766308031496,
 'year_emission_1970': 27.97874547240342,
 'year_emission_1971': 27.166198766247557,
 'year_emission_1972': 29.395352638132483,
 'year_emission_1973': 30.492277257671148,
 'year_emission_1974': 27.705799837396327,
 'year_emission_1975': 30.635980879456177,
 'year_emission_1976': 33.38717102600663,
 'year_emission_1977': 33.38247688815523,
 'year_emission_1978': 32.71382040608911,
 'year_emission_1979': 34.73855554057952,
 'year_emission_1980': 31.89839370095817,
 'year_emission_1981': 31.943179991114263,
 'year_emission_1982': 31.06344076358416,
 'year_emission_1983': 32.12080020525426,
 'year_emission_1984'

 'year_emission_1996': 63.15403051785802,
 'year_emission_1997': 58.08656675296872,
 'year_emission_1998': 58.640204773289526,
 'year_emission_1999': 55.81902529475565,
 'year_emission_2000': 54.67764181757428,
 'year_emission_2001': 55.65038946024732,
 'year_emission_2002': 56.596989347136144,
 'year_emission_2003': 57.17771767171778,
 'year_emission_2004': 56.422470317071905,
 'year_emission_2005': 53.78489001162496,
 'year_emission_2006': 53.61269645963297,
 'year_emission_2007': 52.75637870820379,
 'year_emission_2008': 50.64117800011429,
 'year_emission_2009': 47.068305841618454,
 'year_emission_2010': 52.845024391992375,
 'year_emission_2011': 48.89503448064102,
 'year_emission_2012': 46.32348441050095,
 'year_emission_2013': 44.737617851680355,
 'year_temp_1950': 3.0690000000000004,
 'year_temp_1951': 2.580666666666666,
 'year_temp_1952': 1.779416666666667,
 'year_temp_1953': 3.985583333333333,
 'year_temp_1954': 2.7537499999999997,
 'year_temp_1955': 1.5837500000000004,
 'year_

 'year_temp_1991': 19.924750000000003,
 'year_temp_1992': 20.03316666666667,
 'year_temp_1993': 20.567666666666668,
 'year_temp_1994': 21.266000000000005,
 'year_temp_1995': 20.683166666666665,
 'year_temp_1996': 20.5275,
 'year_temp_1997': 21.084000000000003,
 'year_temp_1998': 20.761333333333337,
 'year_temp_1999': 21.65691666666667,
 'year_temp_2000': 21.140500000000003,
 'year_temp_2001': 21.70125,
 'year_temp_2002': 21.32841666666667,
 'year_temp_2003': 21.47758333333334,
 'year_temp_2004': 20.945083333333333,
 'year_temp_2005': 20.84575,
 'year_temp_2006': 21.310000000000002,
 'year_temp_2007': 21.154666666666667,
 'year_temp_2008': 21.20458333333333,
 'year_temp_2009': 21.0925,
 'year_temp_2010': 21.609750000000002,
 'year_temp_2011': 20.994916666666665,
 'year_temp_2012': 21.479166666666668,
 'year_temp_2013': 19.455222222222222}
{'name': 'Guyana'}
{'name': 'Honduras'}
{'name': 'Croatia',
 'year_emission_1950': 2.5611360000000003,
 'year_emission_1951': 2.927536,
 'year_emissio

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [20]:
# Display updated json_d data
json_d

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': 'AFG',
   'properties': {'name': 'Afghanistan',
    'year_temp_1950': 13.0435,
    'year_emission_1950': 0.084272,
    'year_temp_1951': 13.96775,
    'year_emission_1951': 0.0916,
    'year_temp_1952': 14.175416666666669,
    'year_emission_1952': 0.0916,
    'year_temp_1953': 14.650750000000002,
    'year_emission_1953': 0.10625599999999999,
    'year_temp_1954': 13.691333333333333,
    'year_emission_1954': 0.10625599999999999,
    'year_temp_1955': 14.642583333333334,
    'year_emission_1955': 0.15388800000000002,
    'year_temp_1956': 14.191083333333333,
    'year_emission_1956': 0.1832,
    'year_temp_1957': 12.777166666666666,
    'year_emission_1957': 0.29312,
    'year_temp_1958': 14.716750000000003,
    'year_emission_1958': 0.32976,
    'year_temp_1959': 14.152583333333332,
    'year_emission_1959': 0.38457142047244103,
    'year_temp_1960': 13.985416666666667,
    'year_emission_1960': 0.41388472380163

In [21]:
with open('updated_world.json', 'w') as f:
    f.write(json.dumps(json_d))

In [19]:
# 2015 - 2016 - 2017 -- 

# 2015

# updated map to display field "year_" + <value_returned_from_slider>

